<a href="https://colab.research.google.com/github/nicodes/quant-algos/blob/main/regression/regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
!wget -nc https://lazyprogrammer.me/course_files/sp500_closefull.csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

File ‘sp500_closefull.csv’ already there; not retrieving.



In [39]:
# create stock prices df
df = pd.read_csv('sp500_closefull.csv', index_col=0, parse_dates=True)
df.dropna(axis=0, how='all', inplace=True) # drop any rows where all values are missing
df.dropna(axis=1, how='any', inplace=True) # drop any columns with missing values

df.head()

,CSCO,UAL,TROW,ISRG,PRGO,TPR,DVN,MRO,BA,VRTX,GILD,EQIX,TIF,MDT,V,A,MO,SWKS,MCHP,CDNS,WLTW,EIX,BBY,WBA,AJG,DTE,C,T,CF,DISH,MGM,HUM,WU,APH,SYY,MSI,FCX,ADM,LH,STI,...,AEE,DLTR,HAS,WMT,NTAP,KIM,BAX,LMT,KEY,UNM,BMY,PSA,WYNN,RHI,EFX,NUE,PKG,NBL,CTSH,SWK,MU,TRV,L,AEP,CI,JNJ,WM,DOV,FTI,AGN,M,CRM,PGR,WAT,BWA,LRCX,NWL,UAA,BLK,PPL
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-01-04,24.690001,12.80,54.400002,102.923332,40.349998,36.310001,76.570000,19.153616,56.180000,44.240002,21.650000,109.559998,43.990002,43.900002,22.035000,22.389128,19.790001,14.82,29.180000,6.12,70.913910,34.770000,40.189999,37.299999,22.330000,43.540001,34.000000,28.580000,18.368000,21.180000,9.73,45.680000,19.090000,22.969999,28.100000,32.218723,41.730000,31.469999,76.370003,20.440001,...,27.760000,16.056667,32.540001,54.230000,34.189999,13.66,31.662140,76.849998,5.94,19.940001,25.629999,80.129997,63.959999,27.120001,31.469999,47.790001,23.389999,36.610001,23.400000,52.709999,10.85,49.810001,37.180000,34.939999,36.369999,64.680000,34.160000,28.395418,29.615000,40.290001,17.059999,18.705000,18.030001,61.630001,16.889999,39.880001,15.20,3.51000,238.580002,30.242558
2010-01-05,24.580000,13.91,55.009998,102.459999,38.790001,36.750000,76.650002,19.171511,58.020000,42.779999,21.629999,108.540001,44.279999,44.799999,21.782499,22.145924,19.799999,15.05,28.719999,6.06,70.754967,34.459999,41.209999,37.000000,22.209999,43.049999,35.299999,28.440001,18.666000,21.049999,10.50,45.660000,19.010000,22.490000,27.980000,31.774050,41.980000,31.639999,75.440002,20.719999,...,27.650000,15.853333,32.180000,53.689999,33.590000,13.59,31.444866,77.650002,6.17,20.230000,25.230000,80.790001,67.849998,27.459999,31.459999,48.119999,23.690001,37.560001,23.665001,53.480000,11.17,48.630001,37.380001,34.540001,36.980000,63.930000,34.009998,28.261351,30.125000,39.889999,16.860001,18.625000,17.969999,60.790001,17.695000,39.610001,15.11,3.61500,239.610001,29.851370
2010-01-06,24.420000,13.27,54.150002,103.946663,38.299999,37.470001,76.419998,19.595024,59.779999,42.029999,22.379999,109.529999,44.959999,45.330002,21.490000,22.067240,20.090000,14.91,28.480000,6.13,71.894043,34.889999,40.889999,36.720001,22.219999,43.230000,36.400002,27.610001,19.444000,20.700001,10.14,45.980000,19.000000,22.575001,28.059999,32.218723,43.654999,31.559999,74.190002,21.299999,...,27.459999,16.783333,32.150002,53.570000,33.349998,13.65,31.553503,76.510002,6.13,20.410000,25.219999,80.300003,66.959999,28.719999,31.799999,49.570000,23.570000,37.915001,23.665001,54.820000,11.22,47.939999,37.639999,34.889999,36.900002,64.449997,34.000000,28.596518,30.264999,40.020000,17.100000,18.592501,17.790001,60.900002,18.344999,39.430000,15.38,3.69500,234.669998,29.916569
2010-01-07,24.530001,13.55,54.110001,103.556664,37.990002,37.490002,75.970001,19.475725,62.200001,41.500000,22.434999,107.290001,46.889999,45.750000,21.690001,22.038626,20.059999,14.74,28.330000,6.25,71.682121,34.680000,41.540001,36.939999,22.080000,42.830002,36.500000,27.299999,19.247999,20.790001,10.80,48.119999,19.610001,22.424999,28.090000,32.865524,42.820000,31.230000,74.169998,22.559999,...,27.200001,16.673334,31.690001,53.599998,33.270000,14.11,32.064095,74.410004,6.39,20.860001,25.240000,80.190002,68.389999,29.230000,31.930000,49.230000,23.400000,37.404999,22.770000,56.750000,10.84,48.630001,37.919998,35.189999,38.259998,63.990002,34.080002,29.226633,29.915001,39.700001,17.490000,18.510000,17.549999,61.160000,18.594999,39.360001,15.82,3.65125,237.250000,29.627834
2010-01-08,24.660000,13.33,53.900002,102.986664,37.779999,37.270000,76.120003,19.505550,61.599998,40.669998,22.270000,106.769997,46.009998,45.990002,21.750000,22.031473,20.110001,15.00,28.500000,6.34,71.602646,34.500000,39.910000,36.990002,22.139999,43.160000,35.900002,27.100000,19.662001,21.160000,10.60,47.509998,19.799999,22.590000,27.910000,31.369801,44.049999,30.840000,74.059998,23.010000,...,27.010000,16.443333,31.770000,53.330002,33.540001,13.93,32.140141

In [40]:
# create returns df
df_returns = pd.DataFrame()
for name in df.columns:
  df_returns[name] = np.log(df[name]).diff()
df_returns['SPY'] = df_returns['SPY'].shift(-1)

df_returns['SPY'].tail()

Date
2018-12-21   -0.026778
2018-12-24    0.049290
2018-12-26    0.007648
2018-12-27   -0.001291
2018-12-28         NaN
Name: SPY, dtype: float64

In [41]:
# divide data into train and test sets
test_size = 1000
train = df_returns.iloc[1:-test_size]
test = df_returns.iloc[-test_size:-1]

stock_cols = ['AAPL', 'MSFT', 'AMZN', 'JNJ', 'V', 'PG', 'JPM'] # filter out the rest
Xtrain = train[stock_cols]
Ytrain = train['SPY']
Xtest = test[stock_cols]
Ytest = test['SPY']

# print(Xtrain.head())
# print(Xtest.head())
# print(Ytrain.head())
# print(Ytest.head())

In [42]:
# run regression
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(Xtrain, Ytrain)
model.score(Xtrain, Ytrain), model.score(Xtest, Ytest)

(0.008271754178782342, -0.011369618185063102)

In [43]:
# get directin, round all positive to 1 (buy) all negative to -1 (sell)
Ptrain = model.predict(Xtrain)
Ptest = model.predict(Xtest)
np.mean(np.sign(Ptrain) == np.sign(Ytrain)), np.mean(np.sign(Ptest) == np.sign(Ytest))

print(Ptrain)
print(Ptest)

[-2.37638222e-04  1.05193319e-03  1.02310677e-03 ...  1.47666053e-03
 -8.74157340e-05 -1.40922216e-03]
[ 9.14222685e-04  1.89106910e-03  1.19866719e-04  9.78487796e-04
  2.13901072e-03 -1.64974311e-04  4.92533404e-05  8.30308640e-05
 -2.01042973e-03  8.76547000e-04  5.99085191e-04  3.54836233e-03
 -7.26841991e-05 -1.34614107e-03  7.06045939e-04 -1.13283134e-03
  2.78382442e-04  7.27460613e-04  5.25093412e-05  2.95057680e-04
  4.74344677e-04 -5.77690866e-04 -8.09532353e-05 -1.92567838e-04
 -2.38137895e-04  8.12464650e-04  2.76495145e-04  5.96715633e-04
  1.98343531e-04 -2.08619035e-04  3.08341587e-04  9.47243730e-04
  1.59383560e-04  1.08794014e-03  5.15499046e-04  4.55173389e-04
  8.60382171e-04  7.34493340e-04  1.41257647e-03  1.86290557e-04
  2.03899498e-03  1.11018107e-03 -4.33882832e-05  4.83182838e-04
 -2.87909021e-04 -1.30476799e-04 -2.33089929e-04  1.07246201e-03
  1.53707568e-04  3.77996313e-04  1.05396113e-03  1.88920534e-03
  7.37869693e-04  6.13970916e-04 -8.18844904e-04  1.

In [44]:
df_returns['Position'] = 0
df_returns.loc[1:-test_size,'Position'] = (Ptrain >= 0)
df_returns.loc[-test_size:-1,'Position'] = (Ptest >= 0)
df_returns['AlgoReturn'] = df_returns['Position'] * df_returns['SPY']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [45]:
# Total algo log return train
print(df_returns.iloc[1:-test_size]['AlgoReturn'].sum())

# Total algo log return test
print(df_returns.iloc[-test_size:-1]['AlgoReturn'].sum())

# Total return buy-and-hold train
print(Ytrain.sum())

# Total return buy-and-hold test
print(Ytest.sum())

0.5477174926058384
0.3037759916492
0.5863972019814705
0.19307543946998518
